<a href="https://colab.research.google.com/github/MichalMSlusarski/Basic-sentiment-analysis/blob/main/osint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""OSINT-2.ipynb

Automatically generated by Colaboratory.


"""

# Rozpoznawanie głosu
# Na podstawie: E. Tsukerman: Machine Learning for Cybersecurity Cookbook. Packt Publishing 2019.
# Pliki do wykorzystania: https://commons.wikimedia.org/wiki/Category:BBC_voice_samples

!pip install speechrecognition

import speech_recognition

from google.colab import files
uploaded=files.upload()

file = ("aung.flac", "cooper.flac", "massey.flac")

keywords = ["person", "world", "usa", "freedom", "Cinema", "Kubrick"]

#Definiuje funkcję, która używa Google speech API do konwersji plik audio na tekst.

def transcribe_audio_file_to_text(audio_file):
    recognizer = speech_recognition.Recognizer()
    with speech_recognition.AudioFile(audio_file) as audio_source:
        audio = recognizer.record(audio_source)
        return recognizer.recognize_google(audio)

# Przetwarza plik dźwiękowy na tekst i tworzy słownik wskazujący lokalizację tekstu.
audio_corpus = {}
for audio_file in file:
  audio_corpus[transcribe_audio_file_to_text(audio_file)] = audio_file
print(audio_corpus)

# Identyfikuje zdefiniowane wcześniej słowa kluczowe w przekonwertowanym pliku.
for keyword in keywords:
    for transcription in audio_corpus:
        if keyword in transcription:
            print(
                "keyword "
                + keyword
                + " found in audio "
                + '"'
                + audio_corpus[transcription]
                + '"'
            )

In [ ]:
# -*- coding: utf-8 -*-
"""OSINT-5.ipynb

Automatically generated by Colaboratory.


"""

# Podobieństwo dokumentów. 
# Na podstawie: P. Deitel, H. Deitel: Python dla programistów. Gliwice 2020.

import spacy

nlp = spacy.load("en_core_web_sm")

from google.colab import files
uploaded=files.upload()

# Podobieństwo wg Spacy

with open('sr_aug_trans.txt') as f:
    hamlet = f.read()

with open('assembly_ai_aug_trans.txt') as f:
    makbet = f.read()

doc1 = nlp(makbet)

doc2 = nlp(hamlet)

doc1.similarity(doc2)

token1 = set(hamlet.split())
token2 = set(makbet.split())

def jaccard_similarity(token1, token2):
  nominator = token1.intersection(token2)
  denominator = token1.union(token2)
  similarity = len(nominator)/len(denominator)
  return(similarity)

similarity = jaccard_similarity(token1, token2)
print(similarity)

In [ ]:
# -*- coding: utf-8 -*-
"""OSINT-3.ipynb

Automatically generated by Colaboratory.


"""

#Rozpoznawanie mowy. Na podstawie: https://towardsdatascience.com/a-step-by-step-guide-to-summarizing-audio-files-in-python-55059bec54a7
# Pliki do wykorzystania: https://commons.wikimedia.org/wiki/Category:BBC_voice_samples

from google.colab import files
uploaded=files.upload()

file = ("aung.flac")

import requests
import json
from time import sleep

# Wymaga klucza API - https://www.assemblyai.com/
API_key = "21fd77bc01dd4dc6bde0f0e051ac206e" 

headers = headers = {
    'authorization': API_key, 
    'content-type': 'application/json',
}

upload_endpoint = 'https://api.assemblyai.com/v2/upload'
transcription_endpoint = "https://api.assemblyai.com/v2/transcript"

# Funkcja definiująca wgrywanie plików audio na servery Assembly
def upload(file):

    def read_audio(file):

        with open(file, 'rb') as f:
            while True:
                data = f.read(5_242_880)
                if not data:
                    break
                yield data

    upload_response =  requests.post(upload_endpoint, 
                                     headers=headers, 
                                     data=read_audio(file))

    return upload_response.json().get('upload_url')

# Metoda transcribe()przyjmuje jako argument upload_url i tworzy żądanie POST w celu uzyskania unikalnego identyfikatora transcription_id dla naszego żądania.
def transcribe(upload_url): 

    json = {"audio_url": upload_url, "auto_chapters": True}
    
    response = requests.post(transcription_endpoint, json=json, headers=headers)
    transcription_id = response.json()['id']

    return transcription_id

#Aby zobaczyć wynik transkrypcji, należy stworzyć żądanie GET dla serwerów AssemblyAI. 
#Aby poznać status żądania, należy podać unikalny identyfikator (transcription_id). 

def get_result(transcription_id): 

    current_status = "queued"

    endpoint = f"https://api.assemblyai.com/v2/transcript/{transcription_id}"

    while current_status not in ("completed", "error"):
        
        response = requests.get(endpoint, headers=headers)
        current_status = response.json()['status']
        
        if current_status in ("completed", "error"):
            return response.json()
        else:
            sleep(10)

# Wykonanie żądania
upload_url = upload("cooper.flac")

transcription_id = transcribe(upload_url)

response = get_result(transcription_id)

response

print(response["text"])

In [ ]:
# -*- coding: utf-8 -*-
"""OSINT-7.ipynb

Automatically generated by Colaboratory.


"""

# Ocena czytelności tekstu, https://pypi.org/project/textatistic/

!pip install textatistic

import textatistic

from textatistic import Textatistic # Zainicjowanie obiektu Textatistic

tekst = ("Well, I think storytelling isn't is evolving. The idea of what that even means. We tell a story in a different way than we did, maybe in an oral tradition. Only I think people probably said that about this. The idea of talkies as opposed to a silent film, is it going to be ruined, you know? But I think this the dawn of virtual reality or augmented reality in an experience of a story of cinema, I think is wonderfully exciting.")

Textatistic(text).flesch_score